In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pylab
import warnings
warnings.filterwarnings("ignore")

# 1. Dataset “Babyboom”

In [3]:
colspecs = [
    (0, 8),
    (8, 16),
    (16, 24),
    (24, 32)
]

column_names = [
    "Time of birth",
    "Sex",
    "Birth weight in grams",
    "Number of minutes after midnight of each birth"
]

df = pd.read_fwf("/kaggle/input/babyboom/babyboom.dat.txt", colspecs=colspecs, header=None, names=column_names, 
                    dtype={
                        "Time of birth": int,
                        "Sex": int,
                        "Birth weight in grams": int,
                        "Number of minutes after midnight of each birth": int,
                    })

## Проверьте гипотезу, если средний вес девочек такой же, как вес мальчиков. ##

In [7]:
girls = df[df["Sex"] == 1]["Birth weight in grams"]
boys = df[df["Sex"] == 2]["Birth weight in grams"]

stats.ttest_ind(girls, boys)

TtestResult(statistic=-1.5228564442562815, pvalue=0.1352891891054555, df=42.0)

## Проверьте гипотезу, если дисперсия веса девочек такая же, как и веса мальчиков. ##

In [76]:
def f_test(df1, df2):
    var1 = np.var(df1, ddof=1)
    var2 = np.var(df2, ddof=1)
    if var1 > var2:
        f_statistic = var1/var2
        dof1 = len(df1) - 1 # Степени свободы
        dof2 = len(df2) - 1
    else:
        f_statistic = var2/var1
        dof1 = len(df2) - 1
        dof2 = len(df1) - 1
    p_value = stats.f.sf(f_statistic, dof1, dof2)
    return p_value

In [16]:
print(f_test(girls, boys))

pvalue 0.037631309571425006


# 2. Dataset “Euroweight”

In [43]:
column_names = [
    "ID",
    "weight",
    "batch"
]

df = pd.read_csv("/kaggle/input/euroweight/euroweight.txt", delim_whitespace=True, header=None, names=column_names, 
                    dtype={
                        "ID": int,
                        "weight": float,
                        "batch": int
                    })

## Проверить гипотезы о том, что среднее значение веса монеты одинаково в разных пакетах (попарно и все вместе). ##

In [21]:
batches = df['batch'].unique()

# Создание пустой матрицы для хранения p-значений
p_values_matrix = pd.DataFrame(index=batches, columns=batches)

# Попарное сравнение средних
for batch1, batch2 in itertools.combinations(batches, 2):
    # Извлечение весов для каждой группы
    weights_batch1 = df[df['batch'] == batch1]['weight']
    weights_batch2 = df[df['batch'] == batch2]['weight']
    
    # Проведение t-теста
    t_statistic, p_value = stats.ttest_ind(weights_batch1, weights_batch2)
    
    # Заполнение матрицы p-значениями
    p_values_matrix.loc[batch1, batch2] = p_value.round(4)
    p_values_matrix.loc[batch2, batch1] = p_value.round(4)  # Симметрично заполняем матрицу

# Преобразование значений в матрице в числовой формат
p_values_matrix = p_values_matrix.astype(float)

# Вывод результатов
print(p_values_matrix)

        1       2       3       4       5       6       7       8
1     NaN  0.2615  0.0016  0.0001  0.0000  0.1459  0.2653  0.3566
2  0.2615     NaN  0.0000  0.0067  0.0051  0.0104  0.9605  0.0459
3  0.0016  0.0000     NaN  0.0000  0.0000  0.0717  0.0000  0.0290
4  0.0001  0.0067  0.0000     NaN  0.9120  0.0000  0.0040  0.0000
5  0.0000  0.0051  0.0000  0.9120     NaN  0.0000  0.0029  0.0000
6  0.1459  0.0104  0.0717  0.0000  0.0000     NaN  0.0091  0.6323
7  0.2653  0.9605  0.0000  0.0040  0.0029  0.0091     NaN  0.0437
8  0.3566  0.0459  0.0290  0.0000  0.0000  0.6323  0.0437     NaN


Так как распределение является нормальным, то используем ANOVA

In [26]:
# Создание списка весов для каждой группы
data = [df[df['batch'] == batch]['weight'] for batch in batches]

# Проведение однофакторного ANOVA
stats.f_oneway(*data)

F_onewayResult(statistic=12.67221788627366, pvalue=5.361761521220631e-16)

# 3. Dataset “Iris”

In [65]:
column_names = [
    "sepal length",
    "sepal width",
    "petal length",
    "petal width",
    "class"
]

df = pd.read_csv("/kaggle/input/iris-dataset/iris.txt", header=None, names=column_names, 
                    dtype={
                        "sepal length": float,
                        "sepal width": float,
                        "petal length": float,
                        "petal width": float,
                        "class": str
                    })

## Проверить гипотезы о равенстве распределений характеристик цветков разных типов. ##

In [69]:
classes = df['class'].unique()

# Создаем таблицу для результатов
results = []

# Проходим по каждому признаку
for feature in column_names[:-1]:  # исключаем 'class'
    # Создаем попарные комбинации классов
    combinations = itertools.combinations(classes, 2)
    
    # Проводим тест КС для каждой комбинации
    for class1, class2 in combinations:
        data1 = df[df['class'] == class1][feature]
        data2 = df[df['class'] == class2][feature]
        
        # Проводим тест Колмогорова-Смирнова
        ks_stat, p_value = stats.ks_2samp(data1, data2)
        
        # Добавляем результаты в таблицу
        results.append({
            'Feature': feature,
            'Class 1': class1,
            'Class 2': class2,
            'KS Statistic': ks_stat,
            'p-value': p_value
        })

# Преобразуем результаты в DataFrame
results_df = pd.DataFrame(results)

# Выводим результаты
print(results_df)

         Feature          Class 1          Class 2  KS Statistic       p-value
0   sepal length      Iris-setosa  Iris-versicolor          0.78  2.807571e-15
1   sepal length      Iris-setosa   Iris-virginica          0.92  7.773164e-23
2   sepal length  Iris-versicolor   Iris-virginica          0.46  3.800828e-05
3    sepal width      Iris-setosa  Iris-versicolor          0.68  2.667941e-11
4    sepal width      Iris-setosa   Iris-virginica          0.50  4.807534e-06
5    sepal width  Iris-versicolor   Iris-virginica          0.26  6.779471e-02
6   petal length      Iris-setosa  Iris-versicolor          1.00  1.982331e-29
7   petal length      Iris-setosa   Iris-virginica          1.00  1.982331e-29
8   petal length  Iris-versicolor   Iris-virginica          0.86  3.173228e-19
9    petal width      Iris-setosa  Iris-versicolor          1.00  1.982331e-29
10   petal width      Iris-setosa   Iris-virginica          1.00  1.982331e-29
11   petal width  Iris-versicolor   Iris-virginica  

## Проверьте гипотезы о равенстве средних и дисперсий различных характеристик цветов разных типов ##

Дисперсия

In [77]:
p_values_matrix = pd.DataFrame(index=classes, columns=classes)

# Попарное сравнение дисперсий
for feature in ["sepal length", "sepal width", "petal length", "petal width"]:
    for class1, class2 in itertools.combinations(classes, 2):
        # Извлечение значе# Проведение однофакторного ANOVA
ний для каждой группы
        values_class1 = df[df['class'] == class1][feature]
        values_class2 = df[df['class'] == class2][feature]
        
        # Проведение F-теста (тест на равенство дисперсий)
        p_value = f_test(values_class1, values_class2)
        
        # Заполнение матрицы p-значениями
        p_values_matrix.loc[class1, class2] = p_value.round(4)
        p_values_matrix.loc[class2, class1] = p_value.round(4)  # Симметрично заполняем матрицу

    # Преобразование значений в матрице в числовой формат
    p_values_matrix = p_values_matrix.astype(float)

    # Вывод результатов для текущей характеристики
    print(f"\nLevene's Test Results for {feature}:")
    print(p_values_matrix)


Levene's Test Results for sepal length:
                 Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa              NaN           0.0043          0.0000
Iris-versicolor       0.0043              NaN          0.0739
Iris-virginica        0.0000           0.0739             NaN

Levene's Test Results for sepal width:
                 Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa              NaN           0.0889          0.1233
Iris-versicolor       0.0889              NaN          0.4245
Iris-virginica        0.1233           0.4245             NaN

Levene's Test Results for petal length:
                 Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa              NaN           0.0000          0.0000
Iris-versicolor          0.0              NaN          0.1319
Iris-virginica           0.0           0.1319             NaN

Levene's Test Results for petal width:
                 Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa              NaN    

In [34]:
variance_results = {}

for feature in ["sepal length", "sepal width", "petal length", "petal width"]:
    # Создаем список массивов для каждой группы
    data = [df[df['class'] == cls][feature] for cls in classes]
    
    # Проведение теста Бартлетта
    bartlett_statistic, bartlett_p_value = stats.bartlett(*data)
    
    # Сохраняем результаты
    variance_results[feature] = {
        'Bartlett Statistic': bartlett_statistic,
        'P-Value': bartlett_p_value
    }

# Вывод результатов теста Бартлетта
print("Bartlett's Test Results:")
for feature, result in variance_results.items():
    print(f"{feature}: Bartlett Statistic = {result['Bartlett Statistic']}, P-Value = {result['P-Value']}")

Bartlett's Test Results:
sepal length: Bartlett Statistic = 16.005701874401502, P-Value = 0.0003345076070163035
sepal width: Bartlett Statistic = 2.2158125491551637, P-Value = 0.3302496898960959
petal length: Bartlett Statistic = 55.49409813024258, P-Value = 8.904503355816222e-13
petal width: Bartlett Statistic = 37.99553767573184, P-Value = 5.61531114076804e-09


Среднее

In [35]:
p_values_matrix = pd.DataFrame(index=classes, columns=classes)

# Попарное сравнение средних значений
for feature in ["sepal length", "sepal width", "petal length", "petal width"]:
    for class1, class2 in itertools.combinations(classes, 2):
        # Извлечение значений для каждой группы
        values_class1 = df[df['class'] == class1][feature]
        values_class2 = df[df['class'] == class2][feature]
        
        # Проведение t-теста
        t_statistic, p_value = stats.ttest_ind(values_class1, values_class2)
        
        # Заполнение матрицы p-значениями
        p_values_matrix.loc[class1, class2] = p_value.round(4)
        p_values_matrix.loc[class2, class1] = p_value.round(4)  # Симметрично заполняем матрицу

    # Преобразование значений в матрице в числовой формат
    p_values_matrix = p_values_matrix.astype(float)

    # Вывод результатов для текущей характеристики
    print(f"\nT-Test Results for {feature}:")
    print(p_values_matrix)


T-Test Results for sepal length:
                 Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa              NaN              0.0             0.0
Iris-versicolor          0.0              NaN             0.0
Iris-virginica           0.0              0.0             NaN

T-Test Results for sepal width:
                 Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa              NaN           0.0000          0.0000
Iris-versicolor          0.0              NaN          0.0018
Iris-virginica           0.0           0.0018             NaN

T-Test Results for petal length:
                 Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa              NaN              0.0             0.0
Iris-versicolor          0.0              NaN             0.0
Iris-virginica           0.0              0.0             NaN

T-Test Results for petal width:
                 Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa              NaN              0.0             0.

# 4. Dataset “Surgery”

In [79]:
df = pd.read_csv("/kaggle/input/surgery-dataset/surgery.csv")

In [38]:
df.head()

,V right before,V left before,V right after,V left after
0,7.2,6.70,12.0,13.1
1,1.2,1.20,4.5,4.2
2,6.7,7.30,15.3,14.9
3,9.9,10.05,9.6,9.1
4,3.1,2.13,NaN,NaN


## Проверить гипотезу об успешности операции с вероятностью 0.7 (0.8). Под успехом мы подразумеваем, что «V справа» до операции меньше, чем «V справа» после операции, и одновременно с этим «V слева» до операции меньше, чем «V слева» после операции. ##

In [90]:
# Определение успешных операций
success = (df['V right before'].astype(float) < df['V right after'].astype(float)) & \
          (df['V left before'].astype(float) < df['V left after'].astype(float))

k = success.sum()  # Количество успешных операций
n = len(success)   # Общее количество операций

In [89]:
stats.binomtest(k, n, 0.7, alternative='greater')

BinomTestResult(k=69, n=94, alternative='greater', statistic=0.7340425531914894, pvalue=0.27499327391919837)

In [88]:
stats.binomtest(k, n, 0.8, alternative='less')

BinomTestResult(k=69, n=94, alternative='less', statistic=0.7340425531914894, pvalue=0.07424243944462927)